# Stanford Dogs - A Classfication problem

Classification is a fundamental task in machine learning, and the Stanford Dogs Dataset provides a valuable resource for training and evaluating classification models. The dataset consists of images of various dog breeds, each labeled with the corresponding breed.

By leveraging this dataset, we can develop a classification model that can accurately identify the breed of a given dog image. This can have practical applications in areas such as pet identification, animal welfare, and breed-specific research.

To build a classification model using the Stanford Dogs Dataset, we can employ various machine learning techniques, such as convolutional neural networks (CNNs). CNNs are particularly effective for image classification tasks, as they can automatically learn relevant features from the input images.

By training a CNN on the Stanford Dogs Dataset, we can teach the model to recognize distinctive patterns and characteristics of different dog breeds. Once trained, the model can be used to classify new dog images, providing predictions about the breed with a certain level of confidence.

Evaluation of the classification model can be done using metrics such as accuracy, precision, recall, and F1 score. These metrics help assess the model's performance and determine its effectiveness in correctly classifying dog breeds.

Overall, the Stanford Dogs Dataset offers a valuable opportunity to explore and develop classification models for dog breed identification. By leveraging this dataset and employing appropriate machine learning techniques, we can contribute to the field of computer vision and enhance our understanding of dog breeds.

## 00 - Preprocessing ⚙️

The dataset is split into two parts - Images and Annotations. 

The **Images** are pictures of the 120 different dog breeds present in the dataset. 
The **Annotations** are `.xml`-files, which contains information about where the dog is located in the different pictures and what breed it is.

So first of all we need to load all of these informations into Python, so they can be used to train our model.

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import scipy

# Define paths
images_dir = 'images'

# Create the ImageDataGenerator data generator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2, # 20% of the data will be used for validation
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
)

# Load all images to be used for the training set.
train_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

# Load all images to be used for the validation set.
validation_generator = datagen.flow_from_directory(
    images_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)



Found 16508 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


## 01 - Compiling the model 🔧

The next step in the process is to compile the model itself. But before that we have define what **Loss function**, **Optimizer** and **Metrics** we are going to be using on this model.

For the **Loss function** We have a few different options:

(*Name a few different loss functions that would make sense to use for this project.*)

For the **Optizimers** we also have a few different options:
- *Adam*, *SGD*, *RMSProp* etc.

For the **Metrcis** we also have a few different options:
- *Accuarcy*, *PRecision*, *Recall*, *F1 score* etc.


In [28]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

# ? Load pre-trained model, if available
if os.path.exists('model.h5'):
    model = tf.keras.models.load_model('model.h5')

# ? Otherwise, we need to create a new instance of the model.
else:
    # Load the ResNet50 model, pre-trained on ImageNet
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Add custom layers on top of the base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)

    # Define the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Unfreeze the last few layers of the base model
    for layer in base_model.layers[-10:]:
        layer.trainable = True

    # Compile the model
    model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


## 02 - Train the model 🧠

The next step in the process is to train the now compiled model on our data. Here we also have a little exploratory work in figuring out:
- What *batch size* should we use?
- What *number of epochs* should we use?
- Is the model *overfitting* or *underfitting*?



In [29]:

# - Function the limit the number of batches per epoch for faster iterations.
def limit_batches(generator, max_batches):
    while True:
        for i, (x_batch, y_batch) in enumerate(generator):
            if i >= max_batches:
                break
            yield (x_batch, y_batch)

# * Current limits:
max_train_batches = 100 # It's a good starting point, but needs to be adjusted for better results.
max_validation_batches = 25 # It's a good starting point.

# ? Callbacks and their usage

# 1. Reduce learning rate when a metric has stopped improving.
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
# 2. Stop training when a monitored quantity has stopped improving.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
# 3. Save the model after every epoch.
model_checkpoint = ModelCheckpoint('model.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

# ! 1st round of training
history = model.fit(
    limit_batches(train_generator, max_train_batches),
    validation_data=limit_batches(validation_generator, max_validation_batches),
    epochs=20, # Use a small number of epochs to speed up the process (10 epochs = 5 mins on GPU - With validation accuracy of 0.18 after 10 epochs)
    steps_per_epoch=max_train_batches,
    validation_steps=max_validation_batches
)

Epoch 1/20
100/100 [==============================] - 32s 302ms/step - loss: 11.5807 - accuracy: 0.6606 - val_loss: 11.4933 - val_accuracy: 0.6975
Epoch 2/20
100/100 [==============================] - 29s 294ms/step - loss: 11.5497 - accuracy: 0.6506 - val_loss: 11.4829 - val_accuracy: 0.6587
Epoch 3/20
100/100 [==============================] - 29s 289ms/step - loss: 11.5264 - accuracy: 0.6397 - val_loss: 11.4546 - val_accuracy: 0.6525
Epoch 4/20
100/100 [==============================] - 35s 350ms/step - loss: 11.4679 - accuracy: 0.6447 - val_loss: 11.2225 - val_accuracy: 0.7300
Epoch 5/20
100/100 [==============================] - 37s 367ms/step - loss: 11.4119 - accuracy: 0.6544 - val_loss: 11.2120 - val_accuracy: 0.7165
Epoch 6/20
100/100 [==============================] - 32s 317ms/step - loss: 11.3437 - accuracy: 0.6549 - val_loss: 11.2269 - val_accuracy: 0.6850
Epoch 7/20
100/100 [==============================] - 30s 296ms/step - loss: 11.3145 - accuracy: 0.6662 - val_loss: 11

In [30]:
val_loss, val_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

# Save the model
model.save('model.h5')

127/127 [==============================] - 29s 225ms/step - loss: 10.6037 - accuracy: 0.7099
Validation accuracy: 70.99%


## Futher plan!

1. **Choose the model architecture suitable for our problem** 🤔
    - Convolutional Neural Network (CNN - Good with Image data)
    - Recurrent Neural Network (RNN - Good with sequence data)
    - Another type??

2. **Compile our model** 🔧
    - What *Loss function* should we use? - Cross-entropy is used for classification?
    - What *Optimizer* should we use? Adam, SGD, RMSProp etc.
    - What *Metrics* should we use? Accuracy, precision, recall, f1 score etc.

3. **Train the model** ⚙️
    - What *batch size* should we use?
    - What *number of epochs* should we use?
    - Is the model *overfitting* or *underfitting*?

4. **Evalute the model** 📊
    - Is the model performing as we would like? Based upon our selected metrics to be unbiased 😉

5. **Tune Hyperparameter (Optional) - To improve performance** 📈
    - Use grid search or another thing similar to find the best hyperparameters
    - Adjust model layers, units, learning rate etc.

6. **Save the Model (Optional) - But would be smart** 🧠
    - This can be done, so we don't have to run all the code later to get the model up and running!

7. **Use the Model!**